In [1]:
#!/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/bin/python


#Standard Header used on the projects
# %%

dataSize = 'big'  # 'small'

#first the major packages used for math and graphing
import numpy as np

import os as os
import random
import multiprocessing
from joblib import Parallel, delayed

import cv2
from sklearn.model_selection import train_test_split

import datetime

import platform

HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
    

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    if dataSize == 'big':
        folder = '/sciclone/scr10/dchendrickson01/CraneData/'
        imFolder ='/sciclone/scr10/dchendrickson01/BigData/'
    else:
        folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    if dataSize == 'big':
        folder = 'G:\\CraneData\\'
    else:
        folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
    

scales = 500
#img_height , img_width = scales, 200
DoSomeFiles = False

SmoothType = 3  # 0 = none, 1 = rolling average, 2 = low pass filter, 3 = Kalman filter
WaveletToUse = 'beta'

num_cores = multiprocessing.cpu_count() -1
NumberOfFiles = num_cores - 2
GroupSize = NumberOfFiles


files = os.listdir(folder)
if DoSomeFiles: files = random.sample(files,NumberOfFiles)



In [2]:
import CoreFunctions as cf

In [3]:
def resizeImage(FP):
    res = cv2.resize(FP, dsize=(int(np.shape(FP)[0]/2), int(np.shape(FP)[1]/6)), interpolation=cv2.INTER_CUBIC)

    return res

In [4]:
def saveImage(FP, FName):
    cv2.imwrite(imFolder + FName + '.png', FP)
    return 1

In [5]:
%%capture capt

def MakeImageFiles(files):
    numF = np.size(files)
    Keep = np.zeros(numF)
    for i in range(numF):
        if os.path.isfile(imFolder + files[i] + '.png'):
            Keep[i]=1
    Keep = np.array(Keep, dtype=int)   
    files=np.array(files)[Keep]
    
    AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(file) for file in files)
    Flattened = []
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False:
            print(j,AllAccels[j][1])
        else: 
            Flattened.append(AllAccels[j])

    MetaData = []  #np.asarray([],dtype=object)
    DataOnlyMatrix = np.asarray([],dtype=object)
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False or np.shape(AllAccels[j][0][2])[0] != 60000:
            if AllAccels[j][1][4:9] =='Accel':
                print(j,AllAccels[j][1])
        else: 
            for k in range(3):
                MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
                if np.size(DataOnlyMatrix) == 0:
                        DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
                else:
                        DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

    MetaData = np.matrix(MetaData)

    AllAccels = cf.KalmanGroup(DataOnlyMatrix)

    del DataOnlyMatrix

    maxes = np.amax(AllAccels[:,500:], axis = 1)
    mins = np.amin(AllAccels[:,500:], axis = 1)

    Keep = np.zeros(mins.size)
    for i in range(mins.size):
        if i % 3 == 0:
            if maxes[i] > 0.01 and mins[i] < -0.01:
                Keep[i]=1
                Keep[i+1]=1
                Keep[i+2]=1
                #print(i)


    Keep = np.array(Keep, dtype='bool')

    AllAccels = AllAccels[Keep,:]
    MetaData = MetaData[Keep,:]

    MotionsLeft = int(np.shape(AllAccels)[0]/3.0)

    AllFingers =  Parallel(n_jobs=num_cores)(delayed(cf.makeMatrixImages)([AllAccels[i*3],AllAccels[i*3+1],AllAccels[i*3+2]]) for i in range(MotionsLeft))
    del AllAccels

    SmallFingers =  Parallel(n_jobs=num_cores)(delayed(resizeImage)(FP) for FP in AllFingers)
    del AllFingers

    count =  Parallel(n_jobs=num_cores)(delayed(saveImage)(SmallFingers[i], MetaData[i*3,3]) for i in range(MotionsLeft))
    
    return sum(count)

In [6]:
GroupSize = NumberOfFiles

In [7]:


fCount = len(files)
GroupsLeft = int(fCount/GroupSize) + 1

SplitRatio = 1/(GroupsLeft)

RemainingFiles, GroupFiles, x,y = train_test_split(files, range(len(files)), test_size=SplitRatio, shuffle=True, random_state=0)

GroupsLeft -=1

count = MakeImageFiles(GroupFiles)

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scip

In [ ]:
starttime = datetime.datetime.now()
looptime = starttime
i = 1
while GroupsLeft > 0:
    SplitRatio = 1/(GroupsLeft)

    RemainingFiles, GroupFiles, x,y = train_test_split(RemainingFiles, range(len(RemainingFiles)), test_size=SplitRatio, shuffle=True, random_state=0)
       
    GroupsLeft -=1

    count = MakeImageFiles(GroupFiles)
    #saver.restore('model.ckpt')
    tNow = datetime.datetime.now()
    
    print(count,i,GroupsLeft, tNow-starttime, tNow-looptime)
        #saver.save(sess,'model.ckpt')
    i+=1
    looptime = tNow

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

26 1 1453 0:01:15.010332 0:01:15.010332


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


61 2 1452 0:03:54.089644 0:02:39.079312
34 3 1451 0:05:25.750347 0:01:31.660703


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


26 4 1450 0:06:39.185503 0:01:13.435156
41 5 1449 0:08:26.281992 0:01:47.096489
61 6 1448 0:11:00.868271 0:02:34.586279
61 7 1447 0:13:38.311496 0:02:37.443225
61 8 1446 0:16:11.943171 0:02:33.631675
61 9 1445 0:18:51.139038 0:02:39.195867
27 10 1444 0:20:05.368731 0:01:14.229693
21 11 1443 0:21:06.866030 0:01:01.497299
0 12 1442 0:21:12.088717 0:00:05.222687
61 13 1441 0:23:47.960699 0:02:35.871982
61 14 1440 0:26:25.906633 0:02:37.945934
61 15 1439 0:29:02.050149 0:02:36.143516
24 16 1438 0:30:12.097348 0:01:10.047199
32 17 1437 0:31:39.280961 0:01:27.183613
0 18 1436 0:31:44.339554 0:00:05.058593
61 19 1435 0:34:20.421009 0:02:36.081455
0 20 1434 0:34:25.471622 0:00:05.050613
26 21 1433 0:35:38.094141 0:01:12.622519
61 22 1432 0:38:10.143386 0:02:32.049245
22 23 1431 0:39:13.786261 0:01:03.642875
61 24 1430 0:41:48.559731 0:02:34.773470
61 25 1429 0:44:25.168217 0:02:36.608486
0 26 1428 0:44:30.175220 0:00:05.007003
61 27 1427 0:47:04.336107 0:02:34.160887
22 28 1426 0:48:07.604421 